# You Only Look Once (YOLO)

Este componente utiliza o modelo YOLO para classificação usando [Yolov4](https://pypi.org/project/yolov4/). <br>
Está é uma biblioteca que encapsula o modelo yolov4 com algumas variações, podendo utilizar o modelo completo, como também,a versão reduzida.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

In [53]:
# Download weights
import gdown
gdown.download('https://drive.google.com/uc?id=1L-SO373Udc9tPz5yLkgti5IAXFboVhUt', 'yolov4-full.weights', quiet=False)
gdown.download('https://drive.google.com/u/0/uc?id=1GJwGiR7rizY_19c_czuLN8p31BwkhWY5', 'yolov4-tiny.weights', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1L-SO373Udc9tPz5yLkgti5IAXFboVhUt
To: /home/jovyan/tasks/YOLO (Lucas)/yolov4-full.weights
258MB [00:19, 12.9MB/s] 
Downloading...
From: https://drive.google.com/u/0/uc?id=1GJwGiR7rizY_19c_czuLN8p31BwkhWY5
To: /home/jovyan/tasks/YOLO (Lucas)/yolov4-tiny.weights
24.3MB [00:02, 9.84MB/s]


'yolov4-tiny.weights'

In [1]:
%%writefile Model.py
from typing import List, Iterable, Dict, Union

import base64
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from yolov4.tf import YOLOv4
from yolo_utils import decode_yolo_bbox
import joblib

class Model:
    
    def __init__(self):
        
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/yolo.joblib")
        self.names = artifacts["names"]
        self.inference_parameters = artifacts["inference_parameters"]
        
        # Load Model
        is_tiny = self.inference_parameters['yolo_weight_type'] == 'tiny'
        self.yolo = YOLOv4(tiny=is_tiny)
        self.yolo.classes = "coco.names"
        self.yolo.make_model()
        
        # Load weigths
        if self.inference_parameters['yolo_weight_type'] == 'tiny':
            self.yolo.load_weights("yolov4-tiny.weights", weights_type="yolo")
        else:
            self.yolo.load_weights("yolov4-full.weights", weights_type="yolo")
        
    
    def class_names(self):
        
        return ['x_min', 'y_min', 'x_max', 'y_max', 'class', 'probability']
    
    
    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
            
        # Check if data is a bytes
        if isinstance(X, bytes):
            im_bytes = X # Get image bytes
        
        # If not, should be a list or ndarray
        else:
            # Garantee is a ndarray
            X = np.array(X)
            
            # Seek for extra dimension
            if len(X.shape) == 2:
                im_bytes = X[0,0] # Get image bytes
            
            else:
                im_bytes = X[0] # Get image bytes
        
        # Preprocess img bytes to img_arr
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)
        img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        frame = np.array(img).astype(np.uint8)
        
        # How to interpret YOLO bbox: https://stackoverflow.com/questions/52455429/what-does-the-coordinate-output-of-yolo-algorithm-represent
        predictions = self.yolo.predict(frame, 
                              score_threshold=self.inference_parameters['score_threshold'], 
                              iou_threshold=self.inference_parameters['iou_threshold'])
        
        # Compile results
        results = []
        
        for i, prediction in enumerate(predictions):
            
            result = []
            
            # Decode yolo bbox
            encoded_bbox = prediction[:4]
            decoded_bbox = decode_yolo_bbox(frame, encoded_bbox)
            result += list(decoded_bbox)
            
            # Get class name
            result.append(self.names[int(prediction[4])])
            
            # Get probability
            result.append(prediction[5])
            
            # Compile result
            results.append(result)
            
        return np.array(results)

Overwriting Model.py
